In [ ]:
import polars

In [33]:
# Using polars more elegant s3-style access, but fails with GPU engine.  Can add auth for private data + writes
import polars as pl

endpoint = "minio.carlboettiger.info"
storage_options = {
    "endpoint_url": "https://" + endpoint,
    "aws_skip_signature": "true",
}

upper_districts = pl.scan_parquet(
    "s3://public-census/2024/sld/upper/z8/*",
    storage_options=storage_options
)
# error
# upper_districts.head().collect(engine="gpu")


In [7]:
import polars as pl
import pyarrow.fs as fs
import os

# assumes public URLs
def pl_scan_parquet_gpu(base_path, endpoint = os.getenv("AWS_S3_ENDPOINT", "s3.amazonaws.com")):
    s3 = fs.S3FileSystem(endpoint_override = endpoint, anonymous = True)
    file_stats = s3.get_file_info(fs.FileSelector(base_path, recursive = True))
    
    # 3. Construct "Direct HTTP" URLs
    file_urls = [
        f"https://{endpoint}/{info.path}" 
        for info in file_stats 
        if info.is_file and info.path.endswith(".parquet")
    ]
    return pl.scan_parquet(file_urls)



upper_districts = pl_scan_parquet_gpu("public-census/2024/sld/upper/z8/", "minio.carlboettiger.info")
# 4. Scan the list of HTTP URLs

upper_districts.head().collect(engine="gpu")


STATEFP,SLDUST,GEOID,GEOIDFQ,NAMELSAD,LSAD,LSY,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,h8,h0
str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str
"""01""","""002""","""01002""","""610U900US01002""","""State Senate District 2""","""LU""","""2024""","""G5210""","""N""",353315068,1820551,"""+34.7832617""","""-086.7348706""","""8826493243fffff""","""8027fffffffffff"""
"""01""","""002""","""01002""","""610U900US01002""","""State Senate District 2""","""LU""","""2024""","""G5210""","""N""",353315068,1820551,"""+34.7832617""","""-086.7348706""","""88264930cbfffff""","""8027fffffffffff"""
"""01""","""002""","""01002""","""610U900US01002""","""State Senate District 2""","""LU""","""2024""","""G5210""","""N""",353315068,1820551,"""+34.7832617""","""-086.7348706""","""88264932c5fffff""","""8027fffffffffff"""
"""01""","""002""","""01002""","""610U900US01002""","""State Senate District 2""","""LU""","""2024""","""G5210""","""N""",353315068,1820551,"""+34.7832617""","""-086.7348706""","""882649a9e3fffff""","""8027fffffffffff"""
"""01""","""002""","""01002""","""610U900US01002""","""State Senate District 2""","""LU""","""2024""","""G5210""","""N""",353315068,1820551,"""+34.7832617""","""-086.7348706""","""882649ad55fffff""","""8027fffffffffff"""


In [10]:
import ibis

# env vars
import os
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")
url_style = "path"
use_ssl = os.getenv("AWS_HTTPS", "TRUE")
url_compatibility_mode = True

con = ibis.duckdb.connect()


set_secrets = f'''
CREATE OR REPLACE SECRET s3_{key} (
    TYPE S3,
    KEY_ID '',
    SECRET '',
    ENDPOINT '{endpoint}',
    URL_COMPATIBILITY_MODE {url_compatibility_mode},
    URL_STYLE '{url_style}'
);
'''


con.raw_sql(set_secrets)
upper_districts = con.read_parquet("s3://public-census/2024/sld/upper/z8/**")

upper_districts.head().execute()


,STATEFP,SLDUST,GEOID,GEOIDFQ,NAMELSAD,LSAD,LSY,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,h8,h0
0,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,8826493243fffff,8027fffffffffff
1,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,88264930cbfffff,8027fffffffffff
2,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,88264932c5fffff,8027fffffffffff
3,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,882649a9e3fffff,8027fffffffffff
4,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,882649ad55fffff,8027fffffffffff


In [15]:
import ibis

# env vars
import os
endpoint = os.getenv("AWS_S3_ENDPOINT", "minio.carlboettiger.info")
url_style = "path"

con = ibis.duckdb.connect()
set_secrets = f'''
CREATE OR REPLACE SECRET s3_key (
    TYPE S3,
    KEY_ID '',
    SECRET '',
    ENDPOINT '{endpoint}',
    URL_STYLE '{url_style}'
);
'''


con.raw_sql(set_secrets)
upper_districts = con.read_parquet("s3://public-census/2024/sld/upper/z8/*")

upper_districts.head().execute()


,STATEFP,SLDUST,GEOID,GEOIDFQ,NAMELSAD,LSAD,LSY,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,h8,h0
0,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,8826493243fffff,8027fffffffffff
1,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,88264930cbfffff,8027fffffffffff
2,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,88264932c5fffff,8027fffffffffff
3,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,882649a9e3fffff,8027fffffffffff
4,01,002,01002,610U900US01002,State Senate District 2,LU,2024,G5210,N,353315068,1820551,+34.7832617,-086.7348706,882649ad55fffff,8027fffffffffff


DatabaseTable: ibis_read_parquet_cyoperbmrna3xoegngkdjzxj4i
  STATEFP  string
  SLDUST   string
  GEOID    string
  GEOIDFQ  string
  NAMELSAD string
  LSAD     string
  LSY      string
  MTFCC    string
  FUNCSTAT string
  ALAND    int64
  AWATER   int64
  INTPTLAT string
  INTPTLON string
  h8       string
  h0       string